In [ ]:
is_simulation = True

In [ ]:
import numpy as np
import sys
sys.path.insert(0, '..')
from bundle import FPGA_config, evaluate
from bundle.dashboard_mpl import Dashboard

if is_simulation:
    import matplotlib.pyplot as plt
    %matplotlib inline
    from bundle.simu import Simu
    from pyclk import Trace
else:
    from bundle.pynq import PYNQ
    from pynq import Xlnk, Overlay
    xlnk = Xlnk()
    overlay = Overlay('/home/xilinx/overlays/bundle/design_1.bit')

In [ ]:
iter_nb = 4
mem_nb = iter_nb * 2 * 3 * 2
ddr2fpga_nb = iter_nb * 2
fpga2ddr_nb = iter_nb
mem_depth = 128
add_nb = iter_nb
mul_nb = iter_nb

fpga_config = FPGA_config(ddr2fpga_nb, fpga2ddr_nb, iter_nb, mem_nb, mem_depth, add_nb, mul_nb)

if is_simulation:
    fpga = Simu(fpga_config)
else:
    fpga = PYNQ(fpga_config, overlay)

In [ ]:
if is_simulation:
    trace = Trace()
    for i in range(fpga_config.config['iter_nb']):
        trace.add(fpga.u_iter[i].o_raddr)
        trace.add(fpga.u_iter[i].o_waddr)
        trace.add(fpga.u_iter[i].o_wena)
        trace.add(fpga.s_iter_rmem0_i[i])
        trace.add(fpga.s_iter_rmem1_i[i])
        trace.add(fpga.s_iter_wmem_i[i])
    for i in range(fpga_config.config['mem_nb']):
        trace.add(fpga.s_mem_addr[i])
        trace.add(fpga.s_mem_wena[i])
        trace.add(fpga.s_mem_din[i])
        trace.add(fpga.s_mem_dout[i])
    fpga.set_trace(trace)

In [ ]:
size = 1000
a0 = np.arange(size)
a1 = np.ones_like(a0)
a2 = np.zeros_like(a0)

expr = '(a0 + a1 * (a0 + a2)) * a1 + a1'

In [ ]:
db = Dashboard(fpga)

In [ ]:
res = evaluate(expr, fpga, dashboard=db)

In [ ]:
db.show()

In [ ]:
res

In [ ]:
# FPGA runs at 100 MHz
print('In FPGA:')
print(f'{fpga.time * 1e6 / 1e8} us')

In [ ]:
cexpr = compile(expr, 'fakemodule', 'exec')
print('In CPU:')
%timeit exec(cexpr)

In [ ]:
#res_true = bd.evaluate(expr, toFpga=False)
res_true = eval(expr)
res_true

In [ ]:
if np.array_equal(res, res_true):
    print('Success!')
else:
    print('Failed!')

In [ ]:
res = np.array(res)
diff_idx = np.where(res != res_true)
plt.figure(figsize=(15, 5))
#plt.scatter(np.arange(res_true.size), res_true, c='b')
plt.scatter(diff_idx, res[diff_idx], c='r', label='FPGA')
plt.scatter(diff_idx, res_true[diff_idx], c='g', label='NumPy')
#plt.scatter(diff_idx, z_true[diff_idx]-z[diff_idx], c='r')
#plt.xlim(0, size)
#plt.ylim(1, size+1)
plt.legend(loc='upper left')
plt.title('Differences')
plt.show()

In [ ]:
res[diff_idx]

In [ ]:
res_true[diff_idx]

In [ ]:
trace.plot()